In [2]:
#Import necessary packages:
import numpy as np

#Import the data:
#stored separately and imported here
# as a matrix of X and a *column vector* for the y's
from data.data import X_train, y_train